In [ ]:
import lmdb
import torch
import pickle
import tqdm

In [ ]:
data_path = "/media/drive/qm9/processed/data_v3.pt"
# For single .pt file, process directly
db_path = "/media/drive/qm9/processed/data_v3.lmdb"
# Load the pre-made graphs from the .pt file
print(f"Loading graphs from {data_path}")
graphs = torch.load(data_path, map_location="cpu")
# graphs type is tuple
print(f"graphs type: {type(graphs)}")

print(type(graphs[0]))
print(type(graphs[1]))
print(type(graphs[2]))
# indibidual lists of tensors
print("\n \n graphs[0]")
for k, v in graphs[0].items():
    if isinstance(v, torch.Tensor):
        print(k, v.shape)
    else:
        print(k, len(v))

print("\n \n graphs[1]")
for k, v in graphs[1].items():
    print(k, len(v))

print("\n \n graphs[2]")
print(graphs[2])

dataset = torch.get

In [7]:
import torch
from torch_geometric.datasets import QM9
from scdp.model.module import ChgLightningModule
from scdp.data.data import AtomicData, AtomicNumberTable

# Load PyTorch Geometric dataset
dataset = torch.load("/media/drive/qm9/processed/data_v3.pt")

# Function to convert PyTorch Geometric data to AtomicData
def convert_to_atomic_data(data):
    z_table = AtomicNumberTable(list(range(1, 101)))  # Assuming atomic numbers range from 1 to 100

    atom_types = data['z']
    atom_coords = data['pos']
    cell = torch.eye(3)  # Assuming unit cell for simplicity
    chg_density = torch.randn(atom_coords.shape[0])  # Dummy charge density
    origin = torch.zeros(3)
    metadata = "example"

    atomic_data = AtomicData.build_graph_with_vnodes(
        atom_types=atom_types,
        atom_coords=atom_coords,
        cell=cell,
        chg_density=chg_density,
        origin=origin,
        metadata=metadata,
        z_table=z_table,
        device='cuda'
    )
    
    return atomic_data

# Convert the first data point in the dataset
atomic_data = convert_to_atomic_data(dataset[0])

# Load model
model = ChgLightningModule.load_from_checkpoint(checkpoint_path="/media/drive/scdp/share_models/qm9_none_K4L3_beta_2.0")

# Perform inference
model.eval()
with torch.no_grad():
    coeffs, expo_scaling = model.predict_coeffs(atomic_data)
    orbital_densities = model.orbital_inference(atomic_data, coeffs, expo_scaling, atomic_data.n_probe, atomic_data.probe_coords)

print("Predicted Coefficients:", coeffs)
print("Orbital Densities:", orbital_densities)

TypeError: object of type 'NoneType' has no len()

In [ ]:
import torch
import torch_geometric.data as data

edge_cumulative_counts = graphs[1]["edge_index"]
node_cumulative_counts = graphs[1]["x"]
y_cumulative_counts = graphs[1]["y"]

edge_index_all = graphs[0]["edge_index"]
edge_attr_all = graphs[0]["edge_attr"]
x_all = graphs[0]["x"]
y_all = graphs[0]["y"]
pos_all = graphs[0]["pos"]
z_all = graphs[0]["z"]
idx_all = graphs[0]["idx"]

molecules = []

edge_start = 0
node_start = 0
y_start = 0

for i in range(len(edge_cumulative_counts)):
    edge_end = edge_cumulative_counts[i].item()
    node_end = node_cumulative_counts[i].item()
    y_end = y_cumulative_counts[i].item()

    molecule_edge_index = edge_index_all[:, edge_start:edge_end]
    molecule_edge_attr = edge_attr_all[edge_start:edge_end]
    molecule_x = x_all[node_start:node_end]
    molecule_y = y_all[y_start:y_end]
    molecule_pos = pos_all[node_start:node_end]
    molecule_z = z_all[node_start:node_end]
    molecule_idx = idx_all[i : i + 1]  # idx is per molecule.

    molecule = data.Data(
        x=molecule_x,
        edge_index=molecule_edge_index,
        edge_attr=molecule_edge_attr,
        y=molecule_y,
        pos=molecule_pos,
        z=molecule_z,
        idx=molecule_idx,
    )

    molecules.append(molecule)

    edge_start = edge_end
    node_start = node_end
    y_start = y_end

molecules = molecules[1:]

In [67]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

data_class: Data = graphs[2]
dataset: Data = data_class(graphs[0])

loader = DataLoader(dataset, batch_size=1, shuffle=False)

for batch in loader:
    print(batch)

KeyError: 0

In [57]:
# Now 'molecules' contains a list of Data objects
print(molecules[5].__dict__)


{'_edge_attr_cls': <class 'torch_geometric.data.data.DataEdgeAttr'>, '_tensor_attr_cls': <class 'torch_geometric.data.data.DataTensorAttr'>, '_store': {'x': tensor([[0., 1., 0., 0., 0., 6., 0., 0., 0., 0., 2.],
        [0., 0., 0., 1., 0., 8., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]), 'edge_index': tensor([[0, 0, 0, 1, 2, 3],
        [1, 2, 3, 0, 0, 0]]), 'edge_attr': tensor([[0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]]), 'y': tensor([[ 2.1089e+00,  1.4180e+01, -7.2654e+00, -1.1048e+00,  6.1579e+00,
          5.9989e+01,  7.2390e-01, -3.1153e+03, -3.1152e+03, -3.1152e+03,
         -3.1158e+03,  6.4130e+00, -1.5557e+01, -1.5633e+01, -1.5710e+01,
         -1.4764e+01,  2.8549e+02,  3.8982e+01,  3.4299e+01]]), 'pos': tensor([[-0.0140,  1.1802,  0.0078],
        [ 0.0023, -0.0197,  0.0022],
        [ 

In [52]:
if molecules[1].edge_index.numel() > 0:  # Check if there are edges.
    print(molecules[1].edge_index)
    print(molecules[1].edge_attr)
else:
    print("molecule has no edges")

tensor([[0, 0, 0, 0, 1, 2, 3, 4],
        [1, 2, 3, 4, 0, 0, 0, 0]])
tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]])


In [50]:
print(molecules[0].edge_index)
print(molecules[0].edge_attr)
print(molecules[0].x)
print(molecules[0].y)
print(molecules[0].pos)
print(molecules[0].z)
print(molecules[0].idx)

tensor([], size=(2, 0), dtype=torch.int64)
tensor([], size=(0, 4))
tensor([], size=(0, 11))
tensor([], size=(0, 19))
tensor([], size=(0, 3))
tensor([], dtype=torch.int64)
tensor([0])


In [ ]:
data = graphs[0]
len(data["edge_index"][1])

In [ ]:
graphs[2]
from torch_geometric.data import Data

data = Data(
    x=graphs[0]["x"],
    edge_index=graphs[0]["edge_index"],
    edge_attr=graphs[0]["edge_attr"],
    y=graphs[0]["y"],
    pos=graphs[0]["pos"],
)
print(data)

In [ ]:
# print(graphs[1])
graphs = graphs[0]
# Ensure graphs is a list/sequence
if isinstance(graphs, torch.Tensor):
    graphs = [graphs]
elif isinstance(graphs, dict):
    graphs = list(graphs.values())

print(f"Found {len(graphs)} graphs to process")

db = lmdb.open(
    db_path,
    map_size=1099511627776 * 16,  # 16GB
    subdir=False,
    meminit=False,
    map_async=True,
)

# Store each graph in the LMDB
with db.begin(write=True) as txn:
    for idx, graph in enumerate(
        tqdm(graphs, position=pid, desc=f"Processing {len(graphs)} graphs")
    ):
        txn.put(
            f"{idx}".encode("ascii"),
            pickle.dumps(graph, protocol=-1),
        )
    # Store the total length
    txn.put("length".encode("ascii"), pickle.dumps(len(graphs), protocol=-1))

db.sync()
db.close()

print(f"Successfully processed {total_graphs} graphs to {db_path}")

In [ ]:
import numpy as np
from itertools import product
import plotly.graph_objects as go


def create_grid_points(bounds: tuple[float, float], n_points: int = 20) -> np.ndarray:
    """Create a 3D grid of points."""
    x = np.linspace(bounds[0], bounds[1], n_points)
    y = np.linspace(bounds[0], bounds[1], n_points)
    z = np.linspace(bounds[0], bounds[1], n_points)

    grid_points = np.array(list(product(x, y, z)))
    return grid_points, (n_points, n_points, n_points)


def visualize_charges_3d(
    orbital_inference_func: callable,
    batch: Any,
    coeffs: torch.Tensor,
    bounds: tuple[float, float] = (-5, 5),
    n_points: int = 20,
    opacity: float = 0.7,
    threshold: float = 0.1,
) -> None:
    """
    Visualize charges in 3D space using plotly

    Args:
        orbital_inference_func: Function to compute charges
        batch: Batch object containing molecular data
        coeffs: Orbital coefficients
        bounds: Tuple of (min, max) coordinates for visualization
        n_points: Number of points per dimension
        opacity: Opacity of the visualization
        threshold: Minimum absolute charge value to display
    """
    # Create grid points
    grid_points, shape = create_grid_points(bounds, n_points)
    grid_points_tensor = torch.tensor(grid_points, device=coeffs.device)

    # Calculate charges
    charges = orbital_inference_func(
        batch=batch,
        coeffs=coeffs,
        expo_scaling=None,
        n_probe=torch.tensor([len(grid_points)]),
        probe_coords=grid_points_tensor,
    )

    charges = charges.cpu().numpy()
    charges_3d = charges.reshape(shape)

    # Create visualization
    fig = go.Figure(
        data=go.Volume(
            x=grid_points[:, 0],
            y=grid_points[:, 1],
            z=grid_points[:, 2],
            value=charges,
            isomin=threshold,
            isomax=charges.max(),
            opacity=opacity,
            surface_count=17,  # number of isosurfaces
            colorscale="RdBu",  # red for positive, blue for negative
            caps=dict(x_show=False, y_show=False, z_show=False),
        )
    )

    # Add atoms as spheres
    atom_coords = batch.coords.cpu().numpy()
    atom_types = batch.atom_types.cpu().numpy()

    for coord, atom_type in zip(atom_coords, atom_types):
        fig.add_trace(
            go.Scatter3d(
                x=[coord[0]],
                y=[coord[1]],
                z=[coord[2]],
                mode="markers",
                marker=dict(
                    size=10,
                    color=f"rgb{ATOM_COLORS.get(atom_type, (200,200,200))}",
                ),
                name=f"Atom type {atom_type}",
            )
        )

    # Update layout
    fig.update_layout(
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
        title="3D Charge Distribution",
    )

    fig.show()


# Example atom colors (you can customize these)
ATOM_COLORS = {
    1: (255, 255, 255),  # H - White
    6: (128, 128, 128),  # C - Gray
    7: (0, 0, 255),  # N - Blue
    8: (255, 0, 0),  # O - Red
    # Add more as needed
}

In [68]:
import numpy as np
import plotly.graph_objects as go
from typing import Dict, Any, Optional, Tuple
import torch
import lmdb
import pickle


def load_molecule_from_lmdb(
    db_path: str, molecule_name: str
) -> Optional[Dict[str, torch.Tensor]]:
    """
    Load molecular structure from LMDB database by molecule name.

    Args:
        db_path: Path to LMDB database
        molecule_name: Name of molecule to load

    Returns:
        Dictionary containing molecular structure data or None if not found
    """
    env = lmdb.open(
        db_path,
        map_size=1099511627776 * 2,  # 2GB
        subdir=False,
        readonly=True,
        lock=False,
    )

    with env.begin() as txn:
        # Get total number of molecules
        length = pickle.loads(txn.get("length".encode()))

        # Search for molecule by name
        for i in range(length):
            data = pickle.loads(txn.get(f"{i}".encode()))
            if data.metadata == molecule_name:
                return {
                    "coords": data.coords,
                    "atom_types": data.atom_types,
                    "edge_index": data.edge_index,
                }

    return None


def visualize_molecule_and_charges(
    results: Dict[str, Any],
    molecule_data: Dict[str, torch.Tensor],
    iso_min: float = 0.1,
    iso_max: float = 2.0,
    n_isosurfaces: int = 4,
    opacity: float = 0.3,
    show_probes: bool = True,
    colorscale: str = "RdBu",  # Red-Blue diverging colorscale
) -> None:
    """
    Visualize 3D charge distribution and molecular structure.

    Args:
        results: Dictionary containing charge predictions
        molecule_data: Dictionary containing molecular structure
        iso_min: Minimum isosurface value
        iso_max: Maximum isosurface value
        n_isosurfaces: Number of isosurfaces to show
        opacity: Opacity of isosurfaces
        show_probes: Whether to show probe points
        colorscale: Plotly colorscale to use
    """
    # Atom colors and sizes
    ATOM_COLORS = {
        1: "#FFFFFF",  # H - White
        6: "#808080",  # C - Gray
        7: "#0000FF",  # N - Blue
        8: "#FF0000",  # O - Red
        9: "#90E050",  # F - Light Green
        16: "#FFC832",  # S - Yellow
    }

    ATOM_SIZES = {
        1: 0.4,  # H
        6: 0.7,  # C
        7: 0.65,  # N
        8: 0.6,  # O
        9: 0.5,  # F
        16: 1.0,  # S
    }

    # Create figure
    fig = go.Figure()

    # Reshape predictions into a 3D grid, using cube-root of number of predictions
    grid_size = int(np.cbrt(len(results["predictions"])))
    values = results["predictions"].reshape(grid_size, grid_size, grid_size)

    # Add isosurfaces
    iso_values = np.linspace(iso_min, iso_max, n_isosurfaces)
    for iso_val in iso_values:
        fig.add_trace(
            go.Isosurface(
                x=np.arange(grid_size),
                y=np.arange(grid_size),
                z=np.arange(grid_size),
                value=values,
                isomin=iso_val,
                isomax=iso_val,
                surface_count=1,
                colorscale=colorscale,
                opacity=opacity,
                name=f"Density {iso_val:.2f}",
                showscale=True,
                caps=dict(x_show=False, y_show=False, z_show=False),
            )
        )

    # Add atoms
    coords = molecule_data["coords"].cpu().numpy()
    atom_types = molecule_data["atom_types"].cpu().numpy()

    for atom_type in np.unique(atom_types):
        mask = atom_types == atom_type
        fig.add_trace(
            go.Scatter3d(
                x=coords[mask, 0],
                y=coords[mask, 1],
                z=coords[mask, 2],
                mode="markers",
                marker=dict(
                    size=ATOM_SIZES.get(atom_type, 0.5) * 20,
                    color=ATOM_COLORS.get(atom_type, "#CCCCCC"),
                    symbol="circle",
                    line=dict(color="#000000", width=1),
                ),
                name=f"Atom {atom_type}",
            )
        )

    # Add bonds
    edge_index = molecule_data["edge_index"].cpu().numpy()
    for i in range(edge_index.shape[1]):
        start_idx = edge_index[0, i]
        end_idx = edge_index[1, i]

        # Only draw bonds once (avoid duplicates)
        if start_idx < end_idx:
            fig.add_trace(
                go.Scatter3d(
                    x=[coords[start_idx, 0], coords[end_idx, 0]],
                    y=[coords[start_idx, 1], coords[end_idx, 1]],
                    z=[coords[start_idx, 2], coords[end_idx, 2]],
                    mode="lines",
                    line=dict(color="#000000", width=2),
                    showlegend=False,
                )
            )

    # Add probe points if requested
    if show_probes:
        fig.add_trace(
            go.Scatter3d(
                x=np.arange(grid_size).repeat(grid_size * grid_size),
                y=np.tile(np.arange(grid_size).repeat(grid_size), grid_size),
                z=np.tile(np.arange(grid_size), grid_size * grid_size),
                mode="markers",
                marker=dict(
                    size=2,
                    color=values.flatten(),
                    colorscale=colorscale,
                    showscale=True,
                    opacity=0.5,
                ),
                name="Probe points",
            )
        )

    # Update layout
    fig.update_layout(
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
        ),
        title="3D Charge Distribution with Molecular Structure",
        showlegend=True,
        margin=dict(l=0, r=0, t=30, b=0),
    )

    fig.show()


# Example usage:
# Load molecular data from LMDB
db_path = "/media/drive/qm9/lmdb_test/data.0000.lmdb"
results = np.load("../../out4.npz")
molecule_name = results["metadata"][0]  # Get name from results
molecule_data = load_molecule_from_lmdb(db_path, molecule_name)

if molecule_data is not None:
    visualize_molecule_and_charges(
        results,
        molecule_data,
        iso_min=0.05,
        iso_max=1.5,
        n_isosurfaces=6,
        opacity=0.2,
        show_probes=True,
    )
else:
    print(f"Could not find molecule {molecule_name} in database")

Could not find molecule gdb_1 in database


In [ ]:
import numpy as np
import plotly.graph_objects as go
from typing import Dict, Any


def visualize_charge_distribution(
    results: Dict[str, Any],
    iso_min: float = 0.1,
    iso_max: float = 2.0,
    n_isosurfaces: int = 4,
    opacity: float = 0.3,
    show_probes: bool = True,
    colorscale: str = "RdBu",  # Red-Blue diverging colorscale
) -> None:
    """
    Visualize 3D charge distribution from prediction results.

    Args:
        results: Dictionary containing predictions and probe coordinates
        iso_min: Minimum isosurface value
        iso_max: Maximum isosurface value
        n_isosurfaces: Number of isosurfaces to show
        opacity: Opacity of isosurfaces
        show_probes: Whether to show probe points
        colorscale: Plotly colorscale to use
    """
    # Reshape predictions into a 3D grid
    grid_size = int(np.cbrt(len(results["predictions"])))
    values = results["predictions"].reshape(grid_size, grid_size, grid_size)

    # Create the figure
    fig = go.Figure()

    # Add isosurfaces
    iso_values = np.linspace(iso_min, iso_max, n_isosurfaces)

    for iso_val in iso_values:
        fig.add_trace(
            go.Isosurface(
                x=np.arange(grid_size),
                y=np.arange(grid_size),
                z=np.arange(grid_size),
                value=values,
                isomin=iso_val,
                isomax=iso_val,
                surface_count=1,
                colorscale=colorscale,
                opacity=opacity,
                name=f"Density {iso_val:.2f}",
                showscale=True,
                caps=dict(x_show=False, y_show=False, z_show=False),
            )
        )

    # Add probe points if requested
    if show_probes:
        fig.add_trace(
            go.Scatter3d(
                x=np.arange(grid_size).repeat(grid_size * grid_size),
                y=np.tile(np.arange(grid_size).repeat(grid_size), grid_size),
                z=np.tile(np.arange(grid_size), grid_size * grid_size),
                mode="markers",
                marker=dict(
                    size=10,
                    color=values.flatten(),
                    colorscale=colorscale,
                    showscale=True,
                    opacity=0.5,
                ),
                name="Probe points",
            )
        )

    # Update layout
    fig.update_layout(
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5)),
        ),
        title="3D Charge Distribution",
        showlegend=True,
        margin=dict(l=0, r=0, t=30, b=0),
    )

    fig.show()


results = np.load("../../out3.npz")
# To use the visualization:
grid_size = int(np.cbrt(len(results["predictions"])))
print(f"Grid size detected: {grid_size}x{grid_size}x{grid_size}")

visualize_charge_distribution(results, iso_min=0.1, iso_max=2.0, n_isosurfaces=4)

In [ ]:
def plot_charge_slices(
    results: Dict[str, Any], slice_indices: tuple[int, int, int] = None
) -> None:
    """Plot 2D slices through the charge density"""
    grid_size = int(np.cbrt(len(results["predictions"])))
    values = results["predictions"].reshape(grid_size, grid_size, grid_size)

    if slice_indices is None:
        slice_indices = (grid_size // 2, grid_size // 2, grid_size // 2)
        print(f"Slice indices: {slice_indices}")

    fig = go.Figure()

    # Add three orthogonal slices
    fig.add_trace(
        go.Heatmap(
            z=values[slice_indices[0], :, :], colorscale="Viridis", name="YZ plane"
        )
    )

    fig.update_layout(title="Charge Density Slices", width=1000, height=300)

    fig.show()


plot_charge_slices(results)

In [ ]:
from deepchem.molnet import load_qm9

# Load the QM9 dataset
qm9_tasks, qm9_datasets, qm9_splits = load_qm9(data_dir="/media/drive/qm9/qm9_data")

# From Coeffs to Orbital Densities

### scatter function
The scatter function (from scdp.common.utils) performs a reduction operation that groups values from a source tensor based on indices and reduces them into a target tensor. In this case:
torch.ones_like(self.Ls) creates a tensor of ones with the same shape as self.Ls
self.Ls provides the indices for scattering
MAX_L+1 specifies the size of the output tensor
The default reduction is sum, so it counts how many orbitals exist for each L value
This effectively creates a histogram of orbital counts per angular momentum L


### GTOs


In [ ]:
from analyze_rotations import *

